<h1 align="center">AwakeRodent, RABIES Condound Correction Code</h1>

<p align="center">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<i>Marie E Galteau, 2024</i></p>


  <table align="center">
    <tr>
      <td align="center"><img src="https://hpc.nih.gov/images/Singularity.png" alt="image" width="40" /></td>
      <td align="center"><img src="https://surfer.nmr.mgh.harvard.edu/pub/data/tmp/brain.png" alt="image" width="150" /></td>
      <td align="center"><img src="https://fsl.fmrib.ox.ac.uk/fsl/wiki_static/fsl/img/fsl-logo-x2.png" alt="image" width="70" /></td>
    </tr>
  </table>
</div>

In [11]:
# -- Load module for singularity -- 

!module load singularity
!module load freesurfer
!module load fsl

import os 
import subprocess
import pandas as pd
from enum import Enum
from IPython.display import display, Markdown, Latex

# -- !! Init Variables !! -- 
scripts_folder='/home/traaffneu/margal/awake_code/awake/scripts/'
metadata_path ='/home/traaffneu/margal/awake_code/awake/scripts/tables/metadata_rabies.tsv'
#metadata_path ='/home/traaffneu/margal/awake_code/awake/scripts/tables/online-ds_metadata_rabies.tsv'


df = pd.read_csv(metadata_path, sep='\t')
df = df.loc[(df['exclude'] != 'yes')]

print('CHECK Metadata Path:', metadata_path)

CHECK Metadata Path: /home/traaffneu/margal/awake_code/awake/scripts/tables/metadata_rabies.tsv


#### Define RABIES parameters for preprocessing

In [12]:
# --- Function: Confound Correction, as a job on HPC ---

def qsub_confound_correction_rabies(subj_num, specie, BIDS_input, preprocess_outputs, confound_correction_outputs, regressors_outputs, Cmd_B_rat_template_path, TR, regressor, FD_censoring, FD_threshold, smoothing):
    
    singularity_path='/opt/singularity/3.10.3/bin/singularity'
    if not os.path.exists(confound_correction_outputs):os.makedirs(confound_correction_outputs)  

    #-- Create the full command string --
    confound_cmd = "/opt/singularity/3.10.3/bin/singularity run " \
                        f"-B {BIDS_input}:/BIDS_input:ro " \
                        f"-B {preprocess_outputs}:/preprocess_outputs " \
                        f"-B {preprocess_outputs}:/preprocess_outputs/ " \
                        f"-B {confound_correction_outputs}:/confound_correction_outputs " \
                        f"{Cmd_B_rat_template_path} " \
                        "/opt/rabies/0.5.0/rabies.sif -p MultiProc confound_correction /preprocess_outputs/ /confound_correction_outputs/ " \
                            f"--TR {TR} " \
                            "--highpass 0.01 " \
                            "--lowpass 0.1 " \
                            "--edge_cutoff 30 " \
                            f"--conf_list {regressor} mot_6 " \
                            f"--frame_censoring='FD_censoring'={FD_censoring},'FD_threshold'={FD_threshold},'DVARS_censoring'='true','minimum_timepoint'='3' " \
                            f"--smoothing_filter {smoothing} " \
                                
    # -- Submit the job --
    cmd = f'echo "Job running: $(echo \'{confound_cmd}\'| qsub -N Awk-C:{specie},{subj_num} -l nodes=1:ppn=1,mem=256gb,walltime=1:00:00)"'
    subprocess.call(cmd, shell=True)

#### Define variable + Launch RABIES confound correction

In [13]:
# --- Run RABIES: Confound Correction ---
    
for index in range(0, 1):
    
    # -- Define variables --
    subj_num=str(df.iloc[index]['rodent.sub'])[:-2]               #remove the 2 last characters with [:-2]
    ses_num=str(df.iloc[index]['rodent.ses'])[:-2] 
    specie = str(df.iloc[index]['rodent.spiecies']) 
    TR=str(df.iloc[index]['func.TR'])[:-2]
    regressors=str(df.iloc[index]['regressors']) 
    regressor_list = regressors.split(', ') 

    
    if specie == 'Mouse' :
        specie = 'M'
        bids_folder = '01-03_mice'
        template_dir=''
        Cmd_B_rat_template_path=''
        smoothing = '0.3'
        seed_mask_list=["/project/4180000.36/AwakeRodent/scratch/RABIES/template/seed_S1-right_mouse.nii.gz", "/project/4180000.36/AwakeRodent/scratch/RABIES/template/seed_S1-left_mouse.nii.gz"]
    
    else : #'Rat'
        specie = 'R'
        bids_folder='02-04_rats'   
        template_dir='/groupshare/traaffneu/preclinimg/templates/SIGMA_Wistar_Rat_Brain_TemplatesAndAtlases_Version1.1'
        Cmd_B_rat_template_path=f"-B {template_dir}:/template"
        smoothing='0.5'
        seed_mask_list=["/project/4180000.36/AwakeRodent/scratch/RABIES/template/seed_S1-right_rat.nii.gz", "/project/4180000.36/AwakeRodent/scratch/RABIES/template/seed_S1-left_rat.nii.gz"]

    # -- Print scan infos -- 
    scan_infos=(f'Specie: **<span style="color:#FFA07A">{specie}</span>**   \n'
                f'Rodent num: <span style="color:#FF5733">0{subj_num}</span>  \n'     
                f'Regressors: <span style="color:#FF91A4">{regressors}</span>  \n'
                f'Session n°: <span style="color:#45B39D">{ses_num}</span>  \n'
                f'TR: <span style="color:#0099FF">{TR}</span>  \n')
    
    display(Markdown(scan_infos)) 

    # -- Input and output directories -- 
    orig_bids_dir=f'/project/4180000.36/AwakeRodent/bids/{bids_folder}/'
    BIDS_input=f'/project/4180000.36/AwakeRodent/scratch/bids/{bids_folder}/sub-0{subj_num}'
    preprocess_outputs=f'/project/4180000.36/AwakeRodent/scratch/RABIES/preprocessed/sub-0{subj_num}_ses-{ses_num}'
    confound_correction_outputs_base=preprocess_outputs+'/confound_correction/'
       
    # -- Run RABIES Confound coorection, as a job on the HPC --

    for regressor in regressor_list:
        
        regressors_outputs = os.path.join(confound_correction_outputs_base, regressor)
        confound_correction_outputs=regressors_outputs
        
        FD_censoring = "false"
        FD_threshold = "0.05"
        
        qsub_confound_correction_rabies(subj_num, specie, BIDS_input, preprocess_outputs, confound_correction_outputs, regressors_outputs, Cmd_B_rat_template_path, TR, regressor, FD_censoring, FD_threshold, smoothing)    
        print(f"RABIES Confound Correction: {regressor}")
        print('-----------') 
        
        if regressor == "global_signal":
            for FD_threshold in [0.1, 0.5]:                
                
                global_sign_outputs = os.path.join(confound_correction_outputs_base, f"Global_signal_{FD_threshold}")
                confound_correction_outputs=global_sign_outputs
                
                #FD_censoring = f"--'FD_censoring'='true', 'FD_threshold'={FD_threshold}, 'DVARS_censoring'='true', 'minimum_timepoint'='3'"
                FD_censoring = "true"
                
                qsub_confound_correction_rabies(subj_num, specie, BIDS_input, preprocess_outputs, confound_correction_outputs, regressors_outputs, Cmd_B_rat_template_path, TR, regressor, FD_censoring,FD_threshold,  smoothing)

                print(f"RABIES Confound Correction: {regressor}, {FD_threshold}")
                print('-----------') 


Specie: **<span style="color:#FFA07A">M</span>**   
Rodent num: <span style="color:#FF5733">0100100</span>  
Regressors: <span style="color:#FF91A4">WM_signal, CSF_signal, global_signal</span>  
Session n°: <span style="color:#45B39D">1</span>  
TR: <span style="color:#0099FF">1</span>  


Job running: 51041907.dccn-l029.dccn.nl
RABIES Confound Correction: WM_signal
-----------
Job running: 51041908.dccn-l029.dccn.nl
RABIES Confound Correction: CSF_signal
-----------
Job running: 51041909.dccn-l029.dccn.nl
RABIES Confound Correction: global_signal
-----------
Job running: 51041910.dccn-l029.dccn.nl
RABIES Confound Correction: global_signal, 0.1
-----------
Job running: 51041911.dccn-l029.dccn.nl
RABIES Confound Correction: global_signal, 0.5
-----------


#### TRASH

In [45]:
# -- Creates output folders per regressor -- 

preprocess_outputs=f'/project/4180000.36/AwakeRodent/scratch/RABIES/RABIES_preprocess/sub-0{subj_num}_ses-{ses_num}'
confound_correction_outputs=preprocess_outputs+'/confound_correction_outputs/'

regressor_list = regressors.split(', ') 

for regressor in regressor_list:
            
    if regressor == "Global_signal":
        for FD_threshold in [0.1, 0.5]:
            FD_censoring = f" --FD_censoring=true,FD_threshold={FD_threshold},DVARS_censoring=false,minimum_timepoint=3"
        
            global_sign_outputs = os.path.join(confound_correction_outputs, f"Global_signal_{FD_threshold}")
            if not os.path.exists(global_sign_outputs):os.makedirs(global_sign_outputs)           
    else:
        regressors_outputs = os.path.join(confound_correction_outputs, regressor)
        if not os.path.exists(regressors_outputs):os.makedirs(regressors_outputs)  # Create confound_correction_outputs directory


['WM_signal', 'CSF_signal', 'Global_signal']
